In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-127525")
exp = Experiment(workspace=ws, name="myexperiment")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

cpu_cluster_name = "hyper-drive"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Cluster Found!.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                           max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
from azureml.train.hyperdrive.parameter_expressions import choice

# Specify parameter sampler
### YOUR CODE HERE ###

ps = RandomParameterSampling({
    "--C" : uniform(0.1,1),
    "--max_iter" : choice(50,100,150,200) })

# Specify a Policy
### YOUR CODE HERE ###

policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###

#if "training" not in os.listdir():
    #os.mkdir("./training")

est = SKLearn(source_directory = '.', compute_target=cpu_cluster_name, entry_script='train.py')

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
 ### YOUR CODE HERE ###

hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy = policy,
                                primary_metric_name="Accuracy",
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=10,
                                max_concurrent_runs=4)


In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

#experiment = Experiment(workspace=ws, name="Project_1")
hyperdrive_run = exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
print('best run',best_run)
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']
print('Best Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n learning rate:',parameter_values)
#joblib.dump(parameter_values, filename='outputs/best-hyperdrive.joblib')

In [ ]:
best_run.get_file_names()

In [ ]:
joblib.dump(parameter_values, filename='outputs/best-hyperdrive.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
### YOUR DATA OBJECT HERE ###
x, y = clean_data(ds)
train_data=x.join(y)


In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dat,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

experiment = Experiment(ws, 'automl_azure')
automl_run = experiment.submit(automl_config,show_output=True)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
best_run, best_model = automl_run.get_output()

In [ ]:
print(best_run)

In [ ]:
print(best_model)

In [ ]:
print(best_model._final_estimator)

In [ ]:
#Save your best automl model
joblib.dump(best_model, filename='outputs/best-automl.joblib')

In [ ]:
print(ws.get_default_compute_target('CPU'))